In [2]:
from network_models.soundsream_models_and_utils.clip_like.mapping_down.ss_direct_downmapping import SS_Direct_Downmapping_Model
import gc
import importlib

import torch
import sys
from pathlib import Path

module_path = str(Path.cwd().parents[0] / "network_models/soundstream_models_and_utils/encoder")
if module_path not in sys.path:
    sys.path.append(module_path)

device = "cpu"

import network_models.clip.models.ss_encoder_downmapping as ed

importlib.reload(ed)

#encoder = Encoder(vocab_size=1024, embed_dim=512, n_heads=1, ff_dim=0, n_layers=2, dropout=0)
encoder = ed.EncoderDownmapping(embed_dim=512, n_heads=4, ff_dim=2, n_layers=1, dropout=0, output=1024,
                                max_seq_len=175).to(device)

gc.collect()
from network_models.soundsream_models_and_utils.ss_encoded_dataset import ss_encoded_dataset_full

data_set = ss_encoded_dataset_full(
    csvPath="/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/data/allEncodings.pkl",
    device="cuda")



model = SS_Direct_Downmapping_Model(1024, 512*175)

2023-02-16 16:09:27.034335: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-16 16:09:27.590347: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-16 16:09:27.590397: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-16 16:09:27.590401: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [17]:

import numpy as np

indices = [1, 155, 12, 2, 30, 60, 20, 90, 30, 40, 900]

batch = torch.stack([ data_set[155][0][0], data_set[0][0][0], data_set[30][0][0], data_set[90][0][0]])
batch2 = torch.stack([ data_set[1][0][0], data_set[80][0][0], data_set[2][0][0], data_set[1][0][0]])

batch_b = torch.stack([data_set[i][0][0] for i in indices])
batch_emo = [np.argmax(data_set[i][1]) for  i in indices]
batch_emo_ohe = np.argmax([data_set[i][1] for i in indices], axis=1)

print(batch_emo)


[4, 0, 2, 3, 0, 0, 5, 1, 0, 3, 1]


In [27]:
x1 = model(batch)
x2 = model(batch2)

x_new = model(batch_b)

def cosine_matr(x):
    mul1 = torch.matmul(x, x.T)
    mul1 = mul1/(x.norm(dim=-1, keepdim=True) * x.norm(dim=-1, keepdim=True))
    return mul1

mul1 = torch.matmul(x1, x1.T)
mul1 = mul1/(x1.norm(dim=-1, keepdim=True) * x1.norm(dim=-1, keepdim=True))
#print(mul1)
#print(torch.softmax(mul1, dim=1))

#print(x1.norm(dim=-1, keepdim=True))

#x1 = x1 / x1.norm(dim=-1, keepdim=True)
x2 = x2 / x2.norm(dim=-1, keepdim=True)

#print(cosine_matr(x_new))
print(np.equal.outer(batch_emo_ohe, batch_emo_ohe)*1)
#   print((np.equal.outer(batch_emo, batch_emo)*2)-1)


torch.cosine_similarity(x1, x2, dim=1)
# print(x1.flip(dims=[0]))
# print(x1)

[[1 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 1 1 0 0 1 0 0]
 [0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 1 0]
 [0 1 0 0 1 1 0 0 1 0 0]
 [0 1 0 0 1 1 0 0 1 0 0]
 [0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 1]
 [0 1 0 0 1 1 0 0 1 0 0]
 [0 0 0 1 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 0 0 1]]


tensor([0.6315, 0.2639, 0.5309, 0.4868], grad_fn=<SumBackward1>)

In [34]:
x1.split(split_size=len(x1)//2)

(tensor([[ 1.1257, -0.1272,  0.3292,  ..., -1.2398,  1.5788,  0.5068],
         [-0.1415, -0.5052,  1.2873,  ..., -0.7325,  0.8649,  0.2101]],
        grad_fn=<SplitBackward0>),
 tensor([[ 1.5992,  0.3381,  2.0955,  ...,  0.2265,  1.1394, -0.2271],
         [ 0.8858,  0.3328,  1.2692,  ..., -0.4881,  1.2690,  1.1370]],
        grad_fn=<SplitBackward0>))

In [2]:
from network_models.soundsream_models_and_utils.ss_encoded_dataset import ss_encoded_dataset_full

data_set= ss_encoded_dataset_full(
    csvPath="/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks_clip/content/datasets/soundstream_encoded/allEncodings_noInducednoStimuli_0_2_sec_v12_1_basic.pkl", device="cuda")
# data_set= ss_encoded_dataset_full(
#     csvPath="/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/data/encoder_datasets/full_encodings_with_umap.pkl", device="cuda", umap=True)

In [7]:
data_set.encoded_dataset.encodedData["encoded"].iloc[0].shape

torch.Size([1, 512, 10])

In [9]:
ds= data_set.encoded_dataset.encodedData["pca"].to_numpy()

In [40]:
import numpy as np
npds = np.asarray([fp.numpy() for fp in ds])
searchVec = np.asarray([0.972153, 0.208038, -0.180692, -0.192047])


In [41]:
from sklearn.neighbors import KDTree
kdt = KDTree(npds, metric='euclidean')
cv = kdt.query(np.atleast_2d(searchVec), k=1, return_distance=False)[0,0]

In [43]:
cv

5505